# MONTE CARLO SEQUENCIAL

## IMPORTAÇÃO DE BIBLIOTECAS

In [2]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

0.0


## LEITURA DE DADOS

In [3]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

In [4]:
#Deixando os dados em horas e nos formatos conhecidos
#Dados de Linha
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

#Dados de Linha
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760 # 8760 horas em um ano
D_LIN['MTTF'] = 1 / D_LIN['Lambda'] #Tempo médio entre falhas
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

In [5]:
Ng = len(D_GEN)
geradores = np.arange(1,Ng+1)
Nl = len(D_LIN)
linhas = np.arange(1,Nl+1)
Nc = len(D_LOAD)
cargas = np.arange(1,Nc+1)

## FUNÇÕES AUXILIARES

In [238]:
# Função de cálculo do tempo de residência (TR)
def sortear_tempo_residencia(taxa):
    return float(np.random.exponential(1 / taxa))

def calcular_estatisticas(resultados):
    if len(resultados) > 1:
        # Calcular a média
        media = sum(resultados) / len(resultados)
        
        # Calcular a variância (usando a fórmula da variância amostral)
        variancia = sum((x - media) ** 2 for x in resultados) / (len(resultados) - 1)  # ddof=1 para variância amostral
        
        # Calcular a variância dividida pelo tamanho da lista
        variancia_media = variancia / len(resultados)
        
        # Calcular o Beta
        beta = (variancia_media ** 0.5) / media 
        return media, variancia, beta
    else:
        return 1, 1, 1


## ALGORITMO PRINCIPAL MCS

In [ ]:
#Parâmetros principais:
tol = 0.1 #Tolerância para  o critério de parada
NY_MAX = 10 #Máximo número de anos para simular
NY = 0 #Contador de anos



In [220]:
#Parâmetros principais:
tol = 0.1 #Tolerância para  o critério de parada
NY_MAX = 10 #Máximo número de anos para simular
NY = 0 #Contador de anos

#Vetores que armazenam os resultados de cada ano simulado
resultados_LOLP = []
resultados_LOLE = []
resultados_EPNS = []
resultados_EENS = []
resultados_LOLF = []
resultados_LOLD = []

In [246]:
#Parâmetros principais:
tol = 0.01 #Tolerância para  o critério de parada
NY_MAX = 20 #Máximo número de anos para simular
NY = 0 #Contador de anos
convergiu = False
beta = 10

#Vetores que armazenam os resultados de cada ano simulado
resultados_LOLP = []
resultados_LOLE = []
resultados_EPNS = []
resultados_EENS = []
resultados_LOLF = []
resultados_LOLD = []

while not convergiu:
    NY += 1

    # Reinicializar estados dos componentes no início de cada ano
    estados_geradores = np.ones(Ng, dtype=int)
    estados_linhas = np.ones(Nl, dtype=int)

    # Sorteio dos tempos de residência para todos os componentes
    TG = [sortear_tempo_residencia(D_GEN.iloc[k]['Lambda']) for k in range(Ng)]
    TL = [sortear_tempo_residencia(D_LIN.iloc[k]['Lambda']) for k in range(Nl)]

    # Passo e: Inicialização de variáveis no início do ano
    TSIS = 0
    horas_com_corte = 0
    energia_nao_suprida = 0
    eventos_corte = 0
    F_LOLE = 0
    F_EENS = 0
    F_LOLF = 0

    while TSIS < 8760:
        # Passo f: Identificar o componente com menor TG ou TL
        menor_TG = min(TG)
        menor_TL = min(TL)

        if menor_TG < menor_TL:
            i = np.argmin(TG)
            TGi = menor_TG
            componente = 'gerador'
            if estados_geradores[i] == 1:
                estados_geradores[i] = 0
            else:
                estados_geradores[i] = 1
        else:
            i = np.argmin(TL)
            TGi = menor_TL
            componente = 'linha'
            if estados_linhas[i] == 1:
                estados_linhas[i] = 0
            else:
                estados_linhas[i] = 1

        # Passo g: Ajustar TG ou TL se maior que 8760 (1 ano)
        if TGi > 8760:
            TGi = 8760

        # Passo h: Calcular TDUR e atualizar TSIS
        TDUR = TGi - TSIS
        TSIS = TGi

        z_ger = estados_geradores
        z_lin = estados_linhas
        corte_carga = calcular_fpo(z_ger, z_lin, False) #Calcula quantos MW foram cortados com essa configuração de componentes

        if corte_carga > 0:
            horas_com_corte += TDUR
            energia_nao_suprida += corte_carga * TDUR
            eventos_corte += 1

        #Passo j (vem antes do passo i): Sorteio de novo TR para i e atualização de TG
        if componente == 'gerador':
            if estados_geradores[i] == 1:
                TR = sortear_tempo_residencia(D_GEN.iloc[i]['Lambda'])
            else:
                TR = sortear_tempo_residencia(D_GEN.iloc[i]['mu'])
            TG[i] = TGi + TR
        else:
            if estados_linhas[i] == 1:
                TR = sortear_tempo_residencia(D_LIN.iloc[i]['Lambda'])
            else:
                TR = sortear_tempo_residencia(D_LIN.iloc[i]['mu'])
            TL[i] = TGi + TR 
        print(TSIS)

    #Cálculo dos índices anuais
    LOLP = horas_com_corte/8760
    LOLE = horas_com_corte
    EPNS = energia_nao_suprida/8760
    EENS = energia_nao_suprida
    LOLF = eventos_corte
    LOLD = LOLE / eventos_corte if eventos_corte > 0 else 0

    # Armazenar resultados anuais em vetores
    resultados_LOLP.append(LOLP)
    resultados_LOLE.append(LOLE)
    resultados_EPNS.append(EPNS)
    resultados_EENS.append(EENS)
    resultados_LOLF.append(LOLF)
    resultados_LOLD.append(LOLD)
    
    if NY > 2:
        # Cálculo dos índices estimados e seus coeficientes de convergência
        media_LOLP, var_LOLP, beta_LOLP = calcular_estatisticas(resultados_LOLP)
        media_LOLE, var_LOLE, beta_LOLE = calcular_estatisticas(resultados_LOLE)
        media_EPNS, var_EPNS, beta_EPNS = calcular_estatisticas(resultados_EPNS)
        media_EENS, var_EENS, beta_EENS = calcular_estatisticas(resultados_EENS)
        media_LOLF, var_LOLF, beta_LOLF = calcular_estatisticas(resultados_LOLF)
        media_LOLD, var_LOLD, beta_LOLD = calcular_estatisticas(resultados_LOLD)
        # Armazenar os betas em um vetor e seleciona o maior:
        beta = max([beta_LOLP, beta_EPNS])
        print(f"Beta máximo: {beta}")

    print(NY)
    if beta < tol or NY > NY_MAX:
        convergiu = True

66.56029316495423
77.6637465479676
168.96791739863576
187.49266902075453
195.8501518420471
202.9662727719667
208.24834749614985
300.422047806271
319.7196641585285
320.5496278756808
325.94894882385177
371.7181066445969
379.9872975387356
590.8393198236848
604.2244428151764
721.4374292648249
760.067170578049
761.5177868102097
801.2487845226988
831.6339385476454
1106.9521112615953
1111.1650124357004
1127.3373331388145
1129.3637223303028
1162.2465940349327
1215.0735670433946
1229.655530955508
1243.860146417386
1259.7208586212803
1268.6867057673835
1346.1623446125332
1352.0779971054048
1364.737087707056
1366.6285166087534
1398.572336700364
1408.8226791337404
1428.2196139839289
1455.9385290944163
1457.208584345604
1520.3014266528805
1539.9059444950988
1541.5679338807054
1550.673284293259
1552.8419545761785
1594.8769608973614
1596.1653465492147
1619.241684627303
1634.8859202569206
1820.3934544604076
1861.0358873880693
1875.2554972773557
1883.4608163261191
2006.749827755331
2009.5816981134521
2

In [248]:
print(media_EENS,media_EPNS,media_LOLD,media_LOLE,media_LOLP,media_LOLF)

169848.11348351013 19.389054050629007 30.014846200197706 1616.047454901758 0.18448030307097696 54.57142857142857


In [247]:
# Exibir os valores de betas
print(f"Beta LOLP: {beta_LOLP}")
print(f"Beta LOLE: {beta_LOLE}")
print(f"Beta EPNS: {beta_EPNS}")
print(f"Beta EENS: {beta_EENS}")
print(f"Beta LOLF: {beta_LOLF}")
print(f"Beta LOLD: {beta_LOLD}")

# Exibir os vetores de resultados para ver como estão se acumulando
print("\nResultados LOLP:", resultados_LOLP)
print("Resultados LOLE:", resultados_LOLE)
print("Resultados EPNS:", resultados_EPNS)
print("Resultados EENS:", resultados_EENS)
print("Resultados LOLF:", resultados_LOLF)
print("Resultados LOLD:", resultados_LOLD)

Beta LOLP: 0.04906865002151483
Beta LOLE: 0.04906865002151482
Beta EPNS: 0.09266692232768277
Beta EENS: 0.09266692232768274
Beta LOLF: 0.041122535521707214
Beta LOLD: 0.0430477420349926

Resultados LOLP: [0.2573093447594432, 0.17601572943796198, 0.19332953362741184, 0.2512739277208051, 0.1893226802898087, 0.2527516431963904, 0.1477805170125847, 0.145065841310762, 0.23172608245209414, 0.1575772988758128, 0.15367213814276998, 0.1674189318016104, 0.13757173674786824, 0.20052721988928465, 0.19527642436626383, 0.17028231831604534, 0.1347458425896494, 0.12533048813830416, 0.2409654027395539, 0.15694348986796205, 0.18919977320812928]
Resultados LOLE: [2254.0298600927226, 1541.897789876547, 1693.5667145761277, 2201.159606834253, 1658.4666793387241, 2214.1043944003795, 1294.557329030242, 1270.7767698822752, 2029.9204822803447, 1380.37713815212, 1346.167930130665, 1466.5898425821072, 1205.1284139113259, 1756.6184462301335, 1710.621477448471, 1491.6731084485573, 1180.3735810853286, 1097.895076091